In [1]:
import pandas as pd
import os
from collections import OrderedDict

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

# ROOT_PATH = 'C:/Users/HP/Desktop/AI/DemandaDeEmpleo/Actualizado/'
ROOT_PATH = '/data/mcrodriguez/SaturdaysAI_ZGZ/'

# Extracting information

In [3]:
preprocessed_path=ROOT_PATH+'eq_1_demanda_empleo/resources/data/preprocessed/'

# Extracting information from CSV files:
for subdir, dirs, files in os.walk(preprocessed_path):
    df_list=[]
    print('Number of CSV files: ', len(files))
    for file in files:
        file_csv_path=os.path.join(subdir, file)
        # print(file_csv_path)
        preprocessed_data_df  = pd.read_csv(file_csv_path,sep=',',encoding='utf-8',header=None,skiprows=[0])  
        # display(preprocessed_data_df)
        df_list.append(preprocessed_data_df)
    df_data = pd.concat(df_list, axis=0, ignore_index=True)
    df_data.rename({0:'sector',1:'id_job',2:'date',3:'job',4:'city',5:'province',6:'office',7:'content',8:'description',9:'task',10:'requirement',11:'english_level',12:'condition',13:'number'}, axis=1, inplace=True)

print('Number of offers: ', df_data.shape[0])
print('Number of columns: ', df_data.shape[1])

Number of CSV files:  57
Number of offers:  2036
Number of columns:  14


In [4]:
# Removing duplicated rows:
df_data.drop_duplicates(subset='id_job', keep="last", inplace=True)
print('Number of offers sin duplicados: ', df_data.shape[0])
print('Number of columns: ', df_data.shape[1])

Number of offers sin duplicados:  642
Number of columns:  14


In [5]:
# Column names:
column_name_list = df_data.columns.tolist()
print('Column names: ', column_name_list)

Column names:  ['sector', 'id_job', 'date', 'job', 'city', 'province', 'office', 'content', 'description', 'task', 'requirement', 'english_level', 'condition', 'number']


# Analyzing the extracted information

In [6]:
# Analyzing unique values by column:
for column_name in column_name_list:
    print('----------------------------------------')
    print('COLUMN NAME: ', column_name)
    unique_data_list = df_data[column_name].unique().tolist()
    print('Number of unique values: ', len(unique_data_list))
    print('Unique values: ', unique_data_list)    

----------------------------------------
COLUMN NAME:  sector
Number of unique values:  23
Unique values:  ['DOCENCIA E INVESTIGACIÓN', 'INDUSTRIA PESADA Y CONSTRUCCIONES METÁLICAS', 'TURISMO Y HOSTELERÍA', 'INDUSTRIAS DE FABRICACIÓN DE EQUIPOS ELECTROMECÁNICOS', 'INDUSTRIAS QUÍMICAS', 'COMERCIO', 'AUTOMOCIÓN', 'ADMINISTRACIÓN Y OFICINAS', 'TRANSPORTES Y COMUNICACIONES', 'SERVICIOS A LAS EMPRESAS', 'INDUSTRIAS ALIMENTARIAS', 'INDUSTRIAS MANUFACTURERAS DIVERSAS', 'SERVICIOS A LA COMUNIDAD Y PERSONALES', 'INDUSTRIAS DE LA MADERA Y CORCHO', 'INDUSTRIAS TEXTILES', 'INDUSTRIAS GRÁFICAS', 'AGRARIO', 'ARTESANÍA', 'MONTAJE E INSTALACIÓN', 'PRODUCCIÓN, TRANSFORMACIÓN Y DISTRIBUCIÓN DE ENERGÍA Y', 'SANIDAD', 'SEGUROS Y FINANZAS', 'MANTENIMIENTO Y REPARACIÓN']
----------------------------------------
COLUMN NAME:  id_job
Number of unique values:  642
Unique values:  [22022000556, 22022001063, 22022001067, 22022000911, 22022000929, 22021007546, 22022001153, 22022001085, 22021009077, 22021009141, 2

# Preprocessing texts by column

In [7]:
# Cleaning "province" column:
BAD_PROVINCE_LIST = ['CID Puertomingalvo','MERCAZARAGOZA','MORA DE RUBIELOS Mora De Rubielos']

for bad_province in BAD_PROVINCE_LIST:
    df_data['province']=df_data['province'].astype(str).str.replace(bad_province,'')
    df_data['province']=df_data['province'].astype(str).str.strip()
    
df_data['province']=df_data['province'].astype(str).str.replace('HUSCA','HUESCA')
df_data['province']=df_data['province'].astype(str).str.replace('Zaragoza','ZARAGOZA')

print('Unique values: ', df_data['province'].unique().tolist())
df_province = df_data.groupby(['province']).size().reset_index(name='count')
display(df_province)

Unique values:  ['TERUEL', 'HUESCA', 'ZARAGOZA', '']


,province,count
0,,2
1,HUESCA,206
2,TERUEL,72
3,ZARAGOZA,362


In [8]:
# Cleaning "city" column:
BAD_LOCATION_LIST = ['S-MARMITONES', 'S-AJUSTADORES DEL AUTOMÓVIL, EN GENERAL.', 'A-MÓVIL', 'A-MOVIL', 'S-OPERADORES DE HORMIGONERA', 'S-OPERADORES DE GRÚA EN CAMIÓN.', 'S-REPARADORES DE PALETS DE MADERA', 'A- OFICIAL DE PRIMERA', '-EMPLEO PÚBLICO-', 'C@S', 'S-PINTORES DE VEHÍCULOS', '- AJUSTADOR/A DEL AUTOMÓVIL', 'A-ELECTRICISTA Y/O ELECTRÓNICO/A DE AUTOMOCIÓN', 'S-AYUDANTES-FREGAPLATOS', 'A-TELEFONISTA', 'R@S Y AYUDANTES DE COCINA', 'R@', 'S-REPOSTEROS', 'N-CARRETILLERO', 'I@ - TRABAJADOR/A APÍCOLA', '- DEPENDIENTE DE FRUTERÍA', 'A-CARNICERO/A', 'A-CAFETERÍA', 'I@', '- TRABAJADOR/A APÍCOLA', 'S-ELECTRICISTAS', 'A-DEPENDIENTE/A INFORMÁTICA', '- RECAMBISTAS', 'C@ MANTENIMIENTO', 'O-COMERCIALES', '-EMPLEO PÚBLICO-', 'G-MAG, TORNEROS-FRESADORES, MECÁNICOS DE MAQUINARIA PESADA, MECATRÓNICOS, ELECTRICISTAS DE AUTOMOCIÓN', 'S-CRISTALEROS', 'G-MAG', 'N-REPARTO MERCANCIA']
BAD_LOCATION_TUPLE_LIST = [('Alca–iz', 'Alcañiz'), ('A’nsa-Sobrarbe', 'Aínsa-Sobrarbe'), ('BinŽfar', 'Binéfar'), ('Gra–Žn', 'Grañén'), ('H’jar', 'Híjar'), ('La Puebla de AlfindŽn', 'La Puebla de Alfindén'), ('Monz—n', 'Monzón'), ('Sabi–‡nigo', 'Sabiñánigo'), ('Sallent de G‡llego', 'Sallent de Gállego'), ('Villamayor de G‡llego', 'Villamayor de Gállego'), ('Andorra', 'Andorra, Teruel'), ('Andorra, Teruel, Teruel', 'Andorra, Teruel'), ('Alfajar’n', 'Alfajarín'), ('Mazale—n', 'Mazaleón'), ('Caba–as de Ebro', 'Cabañas de Ebro'), ('ATECA', 'Ateca'), ('BARBASTRO', 'Barbastro'), ('HUESCA', 'Huesca'), ('MONZîN', 'Monzón'), ('Sari–ena', 'Sariñena'), ('TARAZONA', 'Tarazona'), ('TERUEL', 'Teruel'), ('ZARAGOZA', 'Zaragoza')]

for bad_location in BAD_LOCATION_LIST:
    df_data['city']=df_data['city'].astype(str).str.replace(bad_location,'')
    
for bad_location_tuple in BAD_LOCATION_TUPLE_LIST:
    df_data['city']=df_data['city'].astype(str).str.replace(bad_location_tuple[0],bad_location_tuple[1])    
    df_data['city']=df_data['city'].astype(str).str.strip()

df_data['city']=df_data['city'].astype(str).str.replace('A-ORDESA','Ordesa')

print('Unique values: ', df_data['city'].unique().tolist())
df_city = df_data.groupby(['city']).size().reset_index(name='count')
display(df_city)

Unique values:  ['Aguaviva', 'Villanúa', 'Zaragoza', 'Utebo', 'Tauste', 'Huesca', 'Binéfar', 'Teruel', 'Graus', 'S', 'Mora De Rubielos', 'Sallent de Gállego', 'Sabiñánigo', 'Albarracín', 'Jaca', 'Tarazona', 'Albalate del Arzobispo', 'Calatayud', 'La Muela', 'Sobradiel', 'Valle de Hecho', 'Barbastro', 'Quinto', 'Figueruelas', 'Pinseque', 'Belchite', 'Benabarre', 'Aínsa-Sobrarbe', 'Fraga', 'Ejea de los Caballeros', 'Monzón', 'La Puebla de Alfindén', 'Andorra, Teruel', 'Zuera', 'Calamocha', 'Terrer', 'Escatrón', 'Villanueva de Gállego', 'Maria de Huerva', 'Broto', 'Alcañiz', 'Villamayor de Gállego', 'Albelda', 'Bielsa', 'Híjar', 'Codos', 'Grañén', 'Borja', 'Santa Cilia', 'Poleñino', 'Sariñena', 'María de Huerva', 'Cadrete', 'Cuarte de Huerva', 'El Burgo de Ebro', 'Orera', 'Alagón', 'Lalueza', 'Monflorite-Lascasas', 'Abiego', 'Luceni', 'Alfajarín', 'Sarrión', 'Cañada de Benatanduz', 'Alcubierre', 'Santa Cruz de la Serós', 'La Sotonera', 'Biescas', 'Panticosa', 'Cariñena', 'Alfamén', 'Munie

/tmp/ipykernel_3087598/1913309860.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_data['city']=df_data['city'].astype(str).str.replace(bad_location,'')


,city,count
0,,1
1,Abiego,1
2,Aguaviva,1
3,Alagón,2
4,Albalate de Cinca,1
5,Albalate del Arzobispo,2
6,Albarracín,3
7,Albelda,2
8,Albentosa,1
9,Albero Alto,1


In [9]:
# Cleaning "office" column:
BAD_OFFICE_LIST = [('Sabi–‡nigo', 'SABIÑANIGO'), ('SABI„ANIGO', 'SABIÑANIGO'), ('Alca–iz', 'ALCAÑIZ'), ('MONZON', 'MONZÓN'), ('CASPE', 'ZARAGOZA-COMPROMISO DE CASPE'), ('ZARAGOZA-COMPROMISO DE', 'ZARAGOZA-COMPROMISO DE CASPE'), ('EJEA', 'EJEA DE LOS CABALLEROS'), ('ANDORRA', 'ANDORRA, TERUEL'), ('ZARAGORA-CENTRO', 'ZARAGOZA-CENTRO')]
for bad_office in BAD_OFFICE_LIST:
    df_data['office']=df_data['office'].astype(str).str.replace(bad_office[0], bad_office[1])
    df_data['office']=(df_data['office'].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' '))
    df_data['office']=df_data['office'].astype(str).str.strip()

print('Unique values: ', df_data['office'].unique().tolist())
df_city = df_data.groupby(['office']).size().reset_index(name='count')
display(df_city)

Unique values:  ['ALCAÑIZ', 'ZARAGOZA-RANILLAS', 'ZARAGOZA-SANTANDER', 'ZARAGOZA-COMPROMISO DE CASPE', 'ZARAGOZA-PARQUE DE LA MEMORIA', 'EJEA DE LOS CABALLEROS', 'HUESCA', 'BINEFAR', 'TERUEL', 'ZARAGOZA-CENTRO', 'MONZÓN', 'SABIÑANIGO', 'JACA', 'BARBASTRO', 'FRAGA', 'CALATAYUD', 'ANDORRA, TERUEL', 'CALAMOCHA', 'TARAZONA', 'CASTELLON-CASTELLDEFELS', 'UTRILLAS']


,office,count
0,ALCAÑIZ,18
1,"ANDORRA, TERUEL",7
2,BARBASTRO,20
3,BINEFAR,14
4,CALAMOCHA,1
5,CALATAYUD,15
6,CASTELLON-CASTELLDEFELS,1
7,EJEA DE LOS CABALLEROS,4
8,FRAGA,13
9,HUESCA,85


# Saving cleaned data

In [10]:
# cleaned_path=ROOT_PATH+'eq_1_demanda_empleo/resources/data/cleaned/df_data.csv'
cleaned_path=ROOT_PATH+'eq_1_demanda_empleo/resources/data/cleaned/df_data.csv'

df_data.to_csv(cleaned_path, index=False)